In [28]:
import sklearn
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from pipeline_classes import PipelinePlotter

In [29]:
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [30]:
pipe = Pipeline([('plotter', PipelinePlotter(plot=False)), ('svc', SVC())])

In [31]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('plotter',
                 <pipeline_classes.PipelinePlotter object at 0x000002B042460EE0>),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [32]:
pipe.score(X_test, y_test)

0.88